In [1]:
!pip install gym pygame optree

In [2]:
!pip install tensorflow==2.12.0 keras==2.12.0 keras-rl2==1.0.5 torch

In [3]:
!pip install stable-baselines3==2.3.2

  Using cached gymnasium-0.29.1-py3-none-any.whl.metadata (10 kB)
Using cached gymnasium-0.29.1-py3-none-any.whl (953 kB)
  Attempting uninstall: gymnasium
    Found existing installation: gymnasium 1.0.0a2
    Uninstalling gymnasium-1.0.0a2:
      Successfully uninstalled gymnasium-1.0.0a2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
shimmy 2.0.0 requires gymnasium>=1.0.0a1, but you have gymnasium 0.29.1 which is incompatible.


In [15]:
!pip install h5py==3.8.0

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 18.6 MB/s eta 0:00:00
  Attempting uninstall: h5py
    Found existing installation: h5py 3.11.0
    Uninstalling h5py-3.11.0:
      Successfully uninstalled h5py-3.11.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tf-keras 2.17.0 requires tensorflow<2.18,>=2.17, but you have tensorflow 2.12.0 which is incompatible.


# Environment Script

In [4]:
import gym
from gym import spaces
import numpy as np

class ClaimProcessorEnv(gym.Env):
    def __init__(self):
        super(ClaimProcessorEnv, self).__init__()

        # Define the action and observation space
        self.action_space = spaces.Discrete(8) # 8 possible actions (one for each step in the process)
        self.observation_space = spaces.Discrete(8) # 8 states representing each step in the process

        # Initial state
        self.state = 0
        self.steps_beyond_done = None

    def reset(self):
        self.state = 0
        self.steps_beyond_done = None
        return self.state

    def step(self, action):
        done = False
        reward = 0

        # Define the correct sequence of steps
        correct_sequence = [0, 1, 2, 3, 4, 5, 6, 7]

        # Check if action is correct
        if action == correct_sequence[self.state]:
            reward = 1
            self.state += 1
            if self.state == len(correct_sequence):
                done = True
        else:
            reward = -1

        return self.state, reward, done, {}

    def render(self, mode='human'):
        pass

    def close(self):
        pass


# Test environment

In [5]:
import gym
from claim_processor_env import ClaimProcessorEnv

# Create environment
env = ClaimProcessorEnv()

# Test the environment
state = env.reset()
done = False
while not done:
    action = env.action_space.sample()  # Random action for testing
    state, reward, done, _ = env.step(action)
    print(f"State: {state}, Reward: {reward}, Done: {done}")


State: 1, Reward: 1, Done: False
State: 1, Reward: -1, Done: False
State: 1, Reward: -1, Done: False
State: 2, Reward: 1, Done: False
State: 2, Reward: -1, Done: False
State: 2, Reward: -1, Done: False
State: 2, Reward: -1, Done: False
State: 3, Reward: 1, Done: False
State: 4, Reward: 1, Done: False
State: 4, Reward: -1, Done: False
State: 4, Reward: -1, Done: False
State: 4, Reward: -1, Done: False
State: 4, Reward: -1, Done: False
State: 4, Reward: -1, Done: False
State: 4, Reward: -1, Done: False
State: 5, Reward: 1, Done: False
State: 5, Reward: -1, Done: False
State: 5, Reward: -1, Done: False
State: 5, Reward: -1, Done: False
State: 5, Reward: -1, Done: False
State: 5, Reward: -1, Done: False
State: 5, Reward: -1, Done: False
State: 5, Reward: -1, Done: False
State: 5, Reward: -1, Done: False
State: 5, Reward: -1, Done: False
State: 6, Reward: 1, Done: False
State: 6, Reward: -1, Done: False
State: 6, Reward: -1, Done: False
State: 6, Reward: -1, Done: False
State: 6, Reward: -1

# Train file

In [19]:
import gym
from claim_processor_env import ClaimProcessorEnv
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten
from keras.optimizers import Adam
from rl.agents.dqn import DQNAgent
from rl.policy import EpsGreedyQPolicy
from rl.memory import SequentialMemory

# Custom Epsilon Decay Callback
from rl.callbacks import Callback

class CustomEpsilonDecayCallback(Callback):
    def __init__(self, initial_eps=1.0, final_eps=0.1, decay_factor=0.995):
        self.initial_eps = initial_eps
        self.final_eps = final_eps
        self.decay_factor = decay_factor
        self.epsilon = initial_eps

    def on_step_end(self, episode_step, logs={}):
        self.epsilon = max(self.final_eps, self.epsilon * self.decay_factor)
        self.model.policy.eps = self.epsilon
        return True

# Create environment
env = ClaimProcessorEnv()
nb_actions = env.action_space.n

# Build a simple model
model = Sequential()
model.add(Flatten(input_shape=(1,) + env.observation_space.shape))
model.add(Dense(24))
model.add(Activation('relu'))
model.add(Dense(24))
model.add(Activation('relu'))
model.add(Dense(nb_actions))
model.add(Activation('linear'))

# Configure and compile the agent
memory = SequentialMemory(limit=50000, window_length=1)
policy = EpsGreedyQPolicy(eps=1.0)
dqn = DQNAgent(model=model, nb_actions=nb_actions, memory=memory, nb_steps_warmup=1000,
               target_model_update=1e-2, policy=policy)

# Create an optimizer instance and pass it to compile
optimizer = Adam(learning_rate=1e-3)
dqn.compile(optimizer=optimizer, metrics=['mae'])

# Train the agent with custom epsilon decay callback
epsilon_decay_callback = CustomEpsilonDecayCallback()
dqn.fit(env, nb_steps=100000, visualize=False, verbose=2, callbacks=[epsilon_decay_callback])

# Save the trained policy
dqn.save_weights('dqn_claim_processor_weights.h5f', overwrite=True)


Training for 100000 steps ...
    63/100000: episode: 1, duration: 0.197s, episode steps:  63, steps per second: 320, episode reward: -47.000, mean reward: -0.746 [-1.000,  1.000], mean action: 3.143 [0.000, 7.000],  loss: --, mae: --, mean_q: --
   423/100000: episode: 2, duration: 0.340s, episode steps: 360, steps per second: 1057, episode reward: -344.000, mean reward: -0.956 [-1.000,  1.000], mean action: 1.858 [0.000, 7.000],  loss: --, mae: --, mean_q: --
   843/100000: episode: 3, duration: 0.434s, episode steps: 420, steps per second: 967, episode reward: -404.000, mean reward: -0.962 [-1.000,  1.000], mean action: 1.221 [0.000, 7.000],  loss: --, mae: --, mean_q: --


/usr/local/lib/python3.10/dist-packages/keras/engine/training_v1.py:2359: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


Streaming output truncated to the last 5000 lines.
 56270/100000: episode: 5195, duration: 0.083s, episode steps:   8, steps per second:  97, episode reward:  8.000, mean reward:  1.000 [ 1.000,  1.000], mean action: 3.500 [0.000, 7.000],  loss: 0.000035, mae: 2.864774, mean_q: 4.300598
 56280/100000: episode: 5196, duration: 0.097s, episode steps:  10, steps per second: 103, episode reward:  6.000, mean reward:  0.600 [-1.000,  1.000], mean action: 3.100 [0.000, 7.000],  loss: 0.000019, mae: 3.037975, mean_q: 4.492248
 56288/100000: episode: 5197, duration: 0.105s, episode steps:   8, steps per second:  76, episode reward:  8.000, mean reward:  1.000 [ 1.000,  1.000], mean action: 3.500 [0.000, 7.000],  loss: 0.000012, mae: 2.883024, mean_q: 4.315826
 56297/100000: episode: 5198, duration: 0.091s, episode steps:   9, steps per second:  99, episode reward:  7.000, mean reward:  0.778 [-1.000,  1.000], mean action: 3.444 [0.000, 7.000],  loss: 0.000012, mae: 3.218909, mean_q: 4.707564
 

# Play File

In [22]:
import gym
from claim_processor_env import ClaimProcessorEnv
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten
from keras.optimizers import Adam
from rl.agents.dqn import DQNAgent
from rl.policy import EpsGreedyQPolicy
from rl.memory import SequentialMemory

# Define the steps in the health insurance claims process
steps = [
    "Patient Verification",
    "Consultation",
    "Test Results",
    "Medicine Dispensation",
    "Claim Pre-Authorization",
    "Claims Submission",
    "Review",
    "Approval"
]

# Create environment
env = ClaimProcessorEnv()
nb_actions = env.action_space.n

# Build a simple model with the same architecture used during training
model = Sequential()
model.add(Flatten(input_shape=(1,) + env.observation_space.shape))
model.add(Dense(24))
model.add(Activation('relu'))
model.add(Dense(24))
model.add(Activation('relu'))
model.add(Dense(nb_actions))
model.add(Activation('linear'))

# Configure the agent
memory = SequentialMemory(limit=50000, window_length=1)
policy = EpsGreedyQPolicy()
dqn = DQNAgent(model=model, nb_actions=nb_actions, memory=memory, nb_steps_warmup=10,
               target_model_update=1e-2, policy=policy)

# Load the trained policy weights
dqn.compile(optimizer=Adam(learning_rate=1e-3), metrics=['mae'])
dqn.load_weights('dqn_claim_processor_weights.h5f')

# Run multiple episodes
num_episodes = 5
for episode in range(num_episodes):
    state = env.reset()
    done = False
    print(f"\nEpisode {episode + 1}")
    while not done:
        action = dqn.forward(state)
        state, reward, done, _ = env.step(action)
        state_name = steps[state] if state < len(steps) else "Unknown"
        print(f"State: {state_name}, Reward: {reward}, Done: {done}")
        env.render()



Episode 1
State: Consultation, Reward: 1, Done: False
State: Test Results, Reward: 1, Done: False
State: Medicine Dispensation, Reward: 1, Done: False
State: Claim Pre-Authorization, Reward: 1, Done: False
State: Claims Submission, Reward: 1, Done: False
State: Review, Reward: 1, Done: False
State: Approval, Reward: 1, Done: False
State: Unknown, Reward: 1, Done: True

Episode 2
State: Consultation, Reward: 1, Done: False
State: Test Results, Reward: 1, Done: False
State: Medicine Dispensation, Reward: 1, Done: False
State: Claim Pre-Authorization, Reward: 1, Done: False
State: Claims Submission, Reward: 1, Done: False
State: Review, Reward: 1, Done: False
State: Approval, Reward: 1, Done: False
State: Unknown, Reward: 1, Done: True

Episode 3
State: Consultation, Reward: 1, Done: False
State: Test Results, Reward: 1, Done: False
State: Medicine Dispensation, Reward: 1, Done: False
State: Claim Pre-Authorization, Reward: 1, Done: False
State: Claims Submission, Reward: 1, Done: False


/usr/local/lib/python3.10/dist-packages/keras/engine/training_v1.py:2359: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


# Visualize file

In [9]:
import pygame
import gym
from claim_processor_env import ClaimProcessorEnv
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten
from keras.optimizers import Adam
from rl.agents.dqn import DQNAgent
from rl.policy import EpsGreedyQPolicy
from rl.memory import SequentialMemory

# Initialize Pygame
pygame.init()

# Create environment
env = ClaimProcessorEnv()
nb_actions = env.action_space.n

# Build a simple model
model = Sequential()
model.add(Flatten(input_shape=(1,) + env.observation_space.shape))
model.add(Dense(16))
model.add(Activation('relu'))
model.add(Dense(16))
model.add(Activation('relu'))
model.add(Dense(nb_actions))
model.add(Activation('linear'))

# Configure the agent
memory = SequentialMemory(limit=50000, window_length=1)
policy = EpsGreedyQPolicy()
dqn = DQNAgent(model=model, nb_actions=nb_actions, memory=memory, nb_steps_warmup=10,
               target_model_update=1e-2, policy=policy)

# Load the trained policy weights
dqn.compile(optimizer=Adam(learning_rate=1e-3), metrics=['mae'])
dqn.load_weights('dqn_claim_processor_weights.h5f')

# Pygame settings
screen_width, screen_height = 500, 500
screen = pygame.display.set_mode((screen_width, screen_height))
pygame.display.set_caption("Health Insurance Claims Processor")

# Define colors
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
RED = (255, 0, 0)
GREEN = (0, 255, 0)
BLUE = (0, 0, 255)

# Define grid parameters
grid_size = 8
cell_size = screen_width // grid_size

# Define steps names
steps = ["Patient Verification", "Consultation", "Test Results", "Medicine Dispensation",
         "Claim Pre-Authorization", "Claims Submission", "Review", "Approval"]

# Main visualization loop
running = True
state = env.reset()
done = False
clock = pygame.time.Clock()

while running and not done:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

    action = dqn.forward(state)
    state, reward, done, _ = env.step(action)

    screen.fill(WHITE)

    # Draw grid
    for x in range(0, screen_width, cell_size):
        for y in range(0, screen_height, cell_size):
            rect = pygame.Rect(x, y, cell_size, cell_size)
            pygame.draw.rect(screen, BLACK, rect, 1)

    # Highlight current state
    x = (state % grid_size) * cell_size
    y = (state // grid_size) * cell_size
    pygame.draw.rect(screen, BLUE, (x, y, cell_size, cell_size))

    # Display step names
    for i, step_name in enumerate(steps):
        font = pygame.font.Font(None, 36)
        text = font.render(step_name, True, BLACK)
        text_rect = text.get_rect(center=((i % grid_size) * cell_size + cell_size // 2,
                                           (i // grid_size) * cell_size + cell_size // 2))
        screen.blit(text, text_rect)

    # Display reward
    reward_text = font.render(f"Reward: {reward}", True, RED if reward < 0 else GREEN)
    screen.blit(reward_text, (10, screen_height - 40))

    pygame.display.flip()
    clock.tick(5)  # Limit to 5 frames per second

pygame.quit()
